# Cvičenie 5: Metodológia trénovania a vyhodnotenia neurónových sietí

Cieľom tohto cvičenia je oboznámiť vás so základnými krokmi, ktoré by ste mali vykonať pri trénovaní neurónových sietí. Tieto kroky platia všeobecne pre všetky metódy umelej inteligencie, ale niektoré úlohy sú špecifické pre neurónové siete. Metodológiu vysvetlíme na klasifikačnej úlohe, na definíciu a trénovanie neurónovej siete budeme používať knižnicu [PyTorch](https://pytorch.org/\). Ak PyTorch ešte nemáte nainštalovaný, nainštalujte si ho podľa [tohto návodu](https://github.com/DominikVranay/neural-networks-course/blob/master/labs/lab00-getting-started.md).

Postup vývoja neurónových sietí vieme rozdeliť do nasledujúcich krokov:
1. predspracovanie údajov
2. návrh siete
3. trénovanie siete
4. vyhodnotenie siete

## 1. Predspracovanie údajov

Predspracovanie údajov je prvý krok, ktorý v sebe zahŕňa hneď niekoľko úloh:
* načítanie datasetu
* výber príznakov
* normalizácia hodnôt
* vektorizácia vstupov a výstupov
* rozdelenie datasetu na trénovaciu, testovaciu a validačnú množinu.

Postup pri predspracovaní údajov ukážeme na Iris datasete. Stiahnite si [dataset s implementáciou neurónovej siete](sources/lab05/lab5.zip).

### 1.1. Načítanie datasetu

Na načítanie datasetu existujú rôzne knižnice pre Python, jedna populárna z nich je knižnica `pandas`. Knižnica dokáže načítať rôzne formátované dáta, napríklad formáty csv, html, json, hdf5 a SQL. Náš dataset vieme načítať priamo zo súboru csv nasledovným spôsobom:

In [1]:
import pandas as pd
dataset = pd.read_csv('iris.csv')
print(dataset)

     SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthcm         Species
0              5.1           3.5            1.4           0.2     Iris-setosa
1              4.9           3.0            1.4           0.2     Iris-setosa
2              4.7           3.2            1.3           0.2     Iris-setosa
3              4.6           3.1            1.5           0.2     Iris-setosa
4              5.0           3.6            1.4           0.2     Iris-setosa
..             ...           ...            ...           ...             ...
145            6.7           3.0            5.2           2.3  Iris-virginica
146            6.3           2.5            5.0           1.9  Iris-virginica
147            6.5           3.0            5.2           2.0  Iris-virginica
148            6.2           3.4            5.4           2.3  Iris-virginica
149            5.9           3.0            5.1           1.8  Iris-virginica

[150 rows x 5 columns]


Načítaný dataset má typ DataFrame. K ľubovoľným stĺpcom sa dostaneme zadaním názvu stĺpca ako index datasetu. Ak chceme zobraziť viac stĺpcov, index musí byť zoznam s názvami týchto stĺpcov.

In [2]:
# select only column SepalLengthCm
print(dataset['SepalLengthCm'])

# select columns SepalLengthCm and SepalWidthCm
print(dataset[['SepalLengthCm', 'SepalWidthCm']])

0      5.1
1      4.9
2      4.7
3      4.6
4      5.0
      ... 
145    6.7
146    6.3
147    6.5
148    6.2
149    5.9
Name: SepalLengthCm, Length: 150, dtype: float64
     SepalLengthCm  SepalWidthCm
0              5.1           3.5
1              4.9           3.0
2              4.7           3.2
3              4.6           3.1
4              5.0           3.6
..             ...           ...
145            6.7           3.0
146            6.3           2.5
147            6.5           3.0
148            6.2           3.4
149            5.9           3.0

[150 rows x 2 columns]


Alternatívne vieme zobraziť stĺpce ako keby boli parametrom objektu dataset, alebo vieme použiť aj poradové číslo stĺpca (znak `:` pred čiarkou vyjadruje všetky riadky).

In [3]:
print(dataset.SepalLengthCm)
print(dataset.iloc[:, 0])

0      5.1
1      4.9
2      4.7
3      4.6
4      5.0
      ... 
145    6.7
146    6.3
147    6.5
148    6.2
149    5.9
Name: SepalLengthCm, Length: 150, dtype: float64
0      5.1
1      4.9
2      4.7
3      4.6
4      5.0
      ... 
145    6.7
146    6.3
147    6.5
148    6.2
149    5.9
Name: SepalLengthCm, Length: 150, dtype: float64


K riadkom pristupujeme cez číselné indexy, pričom dokopy ich môžeme mať až tri. Prvé číslo vyjadruje poradové číslo prvého riadku, druhé číslo poradové číslo posledného riadku (vľavo uzavretý interval, podľa [pravidiel indexovania v Pythone](https://www.digitalocean.com/community/tutorials/how-to-index-and-slice-strings-in-python-3)), a tretie číslo step. Takto vieme napríklad vypísať každý druhý riadok z intervalu 1-10:

In [4]:
print(dataset[0:10:2])

   SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthcm      Species
0            5.1           3.5            1.4           0.2  Iris-setosa
2            4.7           3.2            1.3           0.2  Iris-setosa
4            5.0           3.6            1.4           0.2  Iris-setosa
6            4.6           3.4            1.4           0.3  Iris-setosa
8            4.4           2.9            1.4           0.2  Iris-setosa


Alternatívne môžete použiť aj `loc` funkciu DataFrame-ov (podľa indexu atribútu; druhý index vyjadruje otvorený interval), alebo `iloc` funkciu (podľa poradia; druhý index vyjadruje otvorený interval)

In [5]:
print(dataset.loc[0:9:2])
print(dataset.iloc[0:9:2])

   SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthcm      Species
0            5.1           3.5            1.4           0.2  Iris-setosa
2            4.7           3.2            1.3           0.2  Iris-setosa
4            5.0           3.6            1.4           0.2  Iris-setosa
6            4.6           3.4            1.4           0.3  Iris-setosa
8            4.4           2.9            1.4           0.2  Iris-setosa
   SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthcm      Species
0            5.1           3.5            1.4           0.2  Iris-setosa
2            4.7           3.2            1.3           0.2  Iris-setosa
4            5.0           3.6            1.4           0.2  Iris-setosa
6            4.6           3.4            1.4           0.3  Iris-setosa
8            4.4           2.9            1.4           0.2  Iris-setosa


Indexovanie riadkov a stĺpcov viete aj kombinovať, na poradí nezáleží:

In [6]:
print(dataset[:10:2]['SepalLengthCm'])
print(dataset['SepalLengthCm'][:10:2])

0    5.1
2    4.7
4    5.0
6    4.6
8    4.4
Name: SepalLengthCm, dtype: float64
0    5.1
2    4.7
4    5.0
6    4.6
8    4.4
Name: SepalLengthCm, dtype: float64


Z datasetu viete vybrať iba niektoré riadky aj na základe hodnoty niektorého atribútu použitím `lambda` funkcie. Napríklad, pre všetky riadky, kde hodnota SepalLengthCm je viac ako 5:

In [7]:
print(dataset.loc[lambda df:df.SepalLengthCm > 5, :])

     SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthcm         Species
0              5.1           3.5            1.4           0.2     Iris-setosa
5              5.4           3.9            1.7           0.4     Iris-setosa
10             5.4           3.7            1.5           0.2     Iris-setosa
14             5.8           4.0            1.2           0.2     Iris-setosa
15             5.7           4.4            1.5           0.4     Iris-setosa
..             ...           ...            ...           ...             ...
145            6.7           3.0            5.2           2.3  Iris-virginica
146            6.3           2.5            5.0           1.9  Iris-virginica
147            6.5           3.0            5.2           2.0  Iris-virginica
148            6.2           3.4            5.4           2.3  Iris-virginica
149            5.9           3.0            5.1           1.8  Iris-virginica

[118 rows x 5 columns]


Všetky tieto podmnožiny majú typ `DataFrame`. Ak chcete hodnoty použiť ako zoznam, resp. zoznam zoznamov, musíte pridať `values`:

In [8]:
dataset['SepalLengthCm'][:10:].values

array([5.1, 4.9, 4.7, 4.6, 5. , 5.4, 4.6, 5. , 4.4, 4.9])

### 1.2. Výber príznakov

Pred výberom príznakov potrebujeme získať intuitívne pochopenie datasetu a vzťahov medzi jednotlivými atribútmi a výsledkom klasifikácie. V tomto nám pomôže knižnica `Seaborn`, ktorá slúži na vizualizáciu údajov a využíva knižnicu `matplotlib`.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# set plot style
sns.set(style="ticks")
sns.set_palette("husl")

# create plots over all dataset; for subset use iloc indexing
sns.pairplot(dataset, hue="Species")

# display plots using matplotlib
plt.show()

Uvedený kód namapuje záznamy z datasetu v každom možnom príznakovom priestore vo dvojiciach príznakov.

![Vizualizácia datasetu](https://github.com/DominikVranay/neural-networks-course/blob/master/labs/sources/lab05/5.1-dataset-visualization.png?raw=1)

Z grafov vidíme, že ani jedna kombinácia nám nedá lineárne separovateľný dataset, budeme teda používať všetky príznaky, ktoré vyberieme pomocou knižnice `pandas`:

In [10]:
# split data into input (X - select the first four columns) and output (y - select last column)
X = dataset.iloc[:, :4].values
y = dataset.iloc[:, -1].values

### 1.3. Normalizácia hodnôt

Normalizácia hodnôt sa používa najmä pre zložitejšie datasety a urýchli proces trénovania neurónových sietí, ktoré lepšie pracujú s dátami z istého intervalu. Počas normalizácie sa číselné hodnoty namapujú zvyčajne na interval 0 až 1.

Neskôr boli vyvinuté špeciálne vrstvy neurónovej siete práve pre normalizáciu, dnes sa skôr používa takto automatizovaný spôsob normalizácie.

### 1.4. Vektorizácia vstupov a výstupov

Kým neurónové siete dokážu spracovať iba číselné hodnoty, skoro všetky datasety obsahujú aj nečíselné údaje (reťazce, kategórie, booleovské hodnoty, atď.). Preto je potrebné, aby sme tieto hodnoty premenili na vektorovú reprezentáciu. Pri vektorizácii upravíme výstupy na formu *n* čísel, kde *n* je počet tried pri klasifikácii. Každý vektor bude obsahovať práve jednu 1 a ostatné hodnoty budú 0, tieto čísla vyjadrujú mieru príslušnosti k jednotlivým triedam.

V našom datasete potrebujeme upraviť očakávaný výstup, ktorý zatiaľ má formu reťazca. Pri vektorizácii vieme využiť `LabelEncoder` z knižnice `scikit-learn`:

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
# transform string labels into number values 0, 1, 2
y1 = encoder.fit_transform(y)
print(y1)

# transform number values into vector representation
Y = pd.get_dummies(y1).values
print(Y)

**Poznámka:** v niektorých prípadoch dokážete reťazce nahradiť jednoduchými číslami, takýto spôsob ale predpokladá, že čísla, ktoré sú blízko sebe vyjadrujú koncepty, ktoré sú veľmi podobné. Napríklad, ak máme stĺpec s hodnotami *low*, *middle*, *high*, tieto hodnoty vieme nahradiť číslami 1, 2 a 3. Rovnaký spôsob ale nemôžeme použiť s hodnotami ako napríklad značky auta: *Škoda* (1), *Audi* (2), *Lada* (3), pretože neurónová sieť by predpokladala, že Lada (3) je viac podobná Audi (2) ako Škodovke (1).

### 1.5. Rozdelenie datasetu na trénovaciu, testovaciu a validačnú množinu

Ďalšou úlohou je rozdelenie množiny na trénovaciu a testovaciu. Na to použijeme ďalšiu funkciu z knižnice `scikit-learn`, a to `train_test_split` ([dokumentácia](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)), ktorá zachová poradie vstupov a výstupov a má tri dôležité parametre:
1. zoznam vstupov
2. zoznam výstupov
3. test_size - veľkosť testovacej množiny medzi 0 a 1 (môžete použiť aj train_size)

Pre opakovateľnosť trénovania je odporúčané používať random seed zadaním parametra `random_state`.

In [12]:
from sklearn.model_selection import train_test_split
import torch

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
X_train = torch.Tensor(X_train)
X_test = torch.Tensor(X_test)
y_train = torch.Tensor(y_train)

Validačná množina sa pri jednoduchých datasetoch až tak často nepoužíva, slúži ako testovacia množina počas fázy trénovania a môže byť použitá ako podmienka pre ukončenie trénovania.

## 2. Návrh siete

Na definíciu siete použijeme knižnicu PyTorch, v ktorej potrebujem tri veci na vytvorenie jednoduchej siete:

1. model - v tomto kroku použijeme jednoduchý feed-forward sekvenčný model ([dokumentácia](https://keras.io/models/sequential/))
2. vrstvy - použijeme iba plne prepojené dense vrstvy ([dokumentácia](https://keras.io/layers/core/#dense))
3. optimalizátor - algoritmus, ktorý nám zadefinuje spôsob trénovania siete; my použijeme optimalizátor Adam ([dokumentácia](https://keras.io/optimizers/#adam))

V PyTorchi môžeme vytvoriť priamo sekvenčný model s vrstvami. Pri definícii vrstiev potrebujeme zadať počet vstupných a výstupných neurónov vo vrstve a aktivačná funkcia je vlastná vrstva. Počet neurónov v poslednej vrstve má zodpovedať formátu výstupu siete.

In [13]:
model = torch.nn.Sequential(
  # TODO: add dense layer with 4, 10 neurons and tanh activation function
  torch.nn.Linear(4,10),
  torch.nn.Tanh(),
  # TODO: add dense layer with 10, 8 neurons and tanh activation function
  torch.nn.Linear(10,8),
  torch.nn.Tanh(),
  # TODO: add dense layer with 8, 6 neurons and tanh activation function
  torch.nn.Linear(8,6),
  torch.nn.Tanh(),
  # TODO: add dense layer with 6, 3 neurons and softmax activation function
  torch.nn.Linear(6,3),
  #torch.Softmax(-1)
)

Pred trénovaním siete ešte potrebujeme zadefinovať spôsob trénovania cez nasledujúce parametre:
* optimizer (optimalizátor)
* loss function/criterion (chybová funkcia)

In [14]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
#criterion = torch.nn.MSELoss()
criterion = torch.nn.CrossEntropyLoss()

## 3. Trénovanie siete

Ak sme spokojní so sieťou, môžeme ju začať trénovať pomocou vlastného trénovacieho loopu.


In [15]:
for epoch in range(700):
    preds = model(X_train)
    loss = criterion(preds, y_train)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if epoch%50 == 0:
        print("Loss:", loss.detach().item(), "accuracy:", (y_train.argmax(-1) == preds.argmax(-1)).sum().item()/len(y_train))

Loss: 1.13427734375 accuracy: 0.30833333333333335
Loss: 1.0766555070877075 accuracy: 0.36666666666666664
Loss: 0.9180667400360107 accuracy: 0.7
Loss: 0.6473805904388428 accuracy: 0.7416666666666667
Loss: 0.5135729908943176 accuracy: 0.8666666666666667
Loss: 0.41171783208847046 accuracy: 0.9333333333333333
Loss: 0.3230969309806824 accuracy: 0.9583333333333334
Loss: 0.24729567766189575 accuracy: 0.975
Loss: 0.19200201332569122 accuracy: 0.975
Loss: 0.15532924234867096 accuracy: 0.9833333333333333
Loss: 0.13116350769996643 accuracy: 0.9833333333333333
Loss: 0.11474630236625671 accuracy: 0.9833333333333333
Loss: 0.1031409278512001 accuracy: 0.9833333333333333
Loss: 0.09457892924547195 accuracy: 0.9833333333333333


## 4. Vyhodnotenie siete

Vyhodnotenie siete pozostáva z dvoch základných úloh: testovanie a vyhodnotenie. Pre testovanie musíme získať predikcie modelu podobne ako pri trénovaní.

In [16]:
y_pred = model(X_test)

Ďalej porovnáme ozajstné výstupy s očakávanými. Keďže výstup má vektorovú reprezentáciu, potrebujeme zistiť pozíciu kde sa nachádza najväčšia hodnota vo vektore. V tomto nám pomôže knižnica `numpy`, ktorú sme zatiaľ nepoužili explicitne, ale podporuje všetky už použité knižnice. Jedná sa o efektívne a optimalizované riešenie práce s poľami.

Pre vyhodnotenie našej siete použijeme konfúznu maticu. Konfúzna matica je tabuľková reprezentácia, kde v riadkoch máme očakávané triedy a v stĺpcoch vypočítané (predikované). V bunkách tabuľky sú uložené počty príkladov klasifikované v danej kombinácii očakávanej a predikovanej triedy. Ideálny klasifikátor bude mať všetky hodnoty po hlavnej diagonále (ďalšie informácie nájdete na [wikipédii](https://en.wikipedia.org/wiki/Confusion_matrix)).

In [17]:
import numpy as np

y_test_class = np.argmax(y_test,axis=1)
y_pred_class = np.argmax(y_pred.detach().numpy(),axis=1)

from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test_class, y_pred_class))

[[11  0  0]
 [ 0 13  0]
 [ 0  0  6]]


Z konfúznej matici potom vieme vypočítať ďalšie metriky, ako presnosť (accuracy), návratnosť (recall) a precizita (precision):

In [18]:
print(classification_report(y_test_class, y_pred_class))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00         6

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



Presnosť popisuje samotný klasifikátor a vypočíta sa nasledovne:

$ACC = \frac{TP + TN}{P + N}$

kde TP + TN je suma správne klasifikovaných príkladov (na hlavnej diagonále) a P + N je počet všetkých príkladov.

Návratnosť a precizita popisujú klasifikátor pre danú triedu, vypočítajú sa nasledovne:

$REC = \frac{TP}{TP + FN}$

$PREC = \frac{TP}{TP + FP}$

kde TP je počet správne klasifikovaných príkladov z danej triedy, P je počet príkadov z danej triedy v testovacej množine a FP je počet príkladov z testovacej množiny nesprávne klasifikovaných do tejto triedy.

Metóda `classification_report` vypočíta ešte hodnotu F1, ktorá je harmonický priemer návratnosti a precizity:

$F1 = 2 \cdot \frac{REC \cdot PREC}{REC + PREC}$